In [9]:
!pip install InstructorEmbedding
!pip install sentence-transformers==2.2.2

  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Using cached huggingface_hub-0.29.1-py3-none-any.whl (468 kB)
Using legacy 'setup.py install' for sentence-transformers, since package 'wheel' is not installed.
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.25.0
    Uninstalling huggingface-hub-0.25.0:
      Successfully uninstalled huggingface-hub-0.25.0
  Running setup.py install for sentence-transformers ... done


In [30]:
# import packages
import pandas as pd
import numpy as np
from InstructorEmbedding import INSTRUCTOR

In [35]:
valid_df = pd.read_csv('./cv-valid-dev.csv')
valid_df.head()

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-dev/sample-000000.mp3,be careful with your prognostications said the...,1,0,NaN,NaN,NaN,NaN
1,cv-valid-dev/sample-000001.mp3,then why should they be surprised when they se...,2,0,NaN,NaN,NaN,NaN
2,cv-valid-dev/sample-000002.mp3,a young arab also loaded down with baggage ent...,2,0,NaN,NaN,NaN,NaN
3,cv-valid-dev/sample-000003.mp3,i thought that everything i owned would be des...,3,0,NaN,NaN,NaN,NaN
4,cv-valid-dev/sample-000004.mp3,he moved about invisible but everyone could he...,1,0,fourties,female,england,NaN


In [36]:
valid_df.shape

(4076, 8)

In [37]:
from sklearn.metrics.pairwise import cosine_similarity

HOTWORDS = ['be careful','destroy', 'stranger']
model = INSTRUCTOR('hkunlp/instructor-large')

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
def find_similar_records(record):
    query = [
        ['Represent the hotwords for retrieving sentences with similar word: ', HOTWORDS[0]],
        ['Represent the hotwords for retrieving sentences with similar word: ', HOTWORDS[1]],
        ['Represent the hotwords for retrieving sentences with similar word: ', HOTWORDS[2]]
    ]
    record = [['Represent the record for retrieving sentences with similar words: ', record]*3]

    query_embedding = model.encode(query)
    record_embedding = model.encode(record)
    similarity = cosine_similarity(query_embedding, record_embedding)
    if np.argmax(similarity) >= 0.85:
        return True
    return False

valid_df['similarity'] = valid_df['text'].apply(find_similar_records)

In [49]:
valid_df[valid_df['similarity'] == True]['text'].values

array(['be careful with your prognostications said the stranger',
       'a young arab also loaded down with baggage entered and greeted the englishman',
       'i thought that everything i owned would be destroyed', ...,
       'he was thinking about omens and someone had appeared',
       'but they could never have taught him arabic',
       'just handling them made him feel better'], dtype=object)

In [50]:
valid_df.to_csv('cv-valid-dev.csv', index=False)